In [3]:
import os
import pickle
import numpy as np
np.random.seed(42)
import pandas as pd
import keras
from keras.models import Model
from keras.layers import Embedding, Dense, Dropout, Input, Concatenate
from keras.callbacks import EarlyStopping
from keras.utils import plot_model
from sklearn.model_selection import train_test_split as splt
import matplotlib.pyplot as plt
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

Using TensorFlow backend.


In [4]:
dfy = pd.read_csv("MILANO_wSCORE.csv")
dfx = pd.read_csv("PLIC-milano-processed.csv", sep=";")

/home/webvalley/anaconda3/envs/score/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (14,29,36,44,51,94,104,105,112,113,120,127,130,149,158,168,237,239,240,241,248,249,256,273,298) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# Thresholds are 0 - 0.01, 0.01 - 0.02, 0.02 - 0.05, 0.05 - 1
y = dfy["SCORE"].values/100
y_cat = np.zeros((y.shape[0], 4))
for i, val in enumerate(y):
    if(val < 0.01):
        y_cat[i][0] = 1
    elif(val < 0.02):
        y_cat[i][1] = 1
    elif(val < 0.05):
        y_cat[i][2] = 1
    else:
        y_cat[i][3] = 1

In [6]:
dfx = dfx.select_dtypes(exclude=['object'])
dfx = dfx.drop(labels = ['Unnamed: 0', 'Unnamed: 0.1', 'cod pcs'], axis=1) 
dfx = dfx.dropna(how='any', axis=1)
dfx = dfx.drop([col for col in dfx.columns if "date" in col], axis = 1)

In [7]:
dfxFloats = dfx.select_dtypes(include = 'float')
dfxInt = dfx.select_dtypes(include = 'int')
dfxFloats = dfxFloats.join(dfx['TG']).join(dfx['HDL']).join(dfx['glucose']).join(dfx['ages']).join(dfx['TORQUE AND ANGLE WRENCH'])
dfxInt = dfxInt.drop(['TG', 'TORQUE AND ANGLE WRENCH', 'HDL', 'glucose', 'ages'], axis = 1)
dfxInt = dfxInt[dfxInt.columns[dfxInt.max()>0]]

In [8]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.float_format', '{:20,.2f}'.format)
    pd.set_option('display.max_colwidth', -1)
    print(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')
#print_full(dfxInt.max())
#print_full(dfxFloats.max())

In [9]:
xCat = dfxInt.values
xCont = dfxFloats.values

In [10]:
print(xCat.shape)
print(xCont.shape)

(5780, 95)
(5780, 77)


In [11]:
contBranch = Input(shape=(76,))
catBranch = Input(shape=(98,))

In [23]:
cols = dfxInt.columns
for i, colMax in enumerate(dfxInt.max()):
    if(colMax > 1):
        print(cols[i], colMax)

abortion n 5
alcohols 2
aorta calcifications 2
right corneal arch 2
left corneal arch 2
physical activity 3
physical activity intensity 3
self dyslipidaemia 4
self type dyslipidemia 6
enlarged liver 2
smoking recod 2
learning 4
PLACA 9
steatosis degree 3
steatosis NS 2
thyroid function 2
